https://github.com/partytax/ncei-api-guide

In [42]:
import pandas as pd
import xarray as xr
from datetime import datetime

In [20]:
# This builds a file list of ERA5 monthly mean SST files from 1979-2021 (files are 
# for every year but contain monthly means)
base_url = 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/'
base_url2 = '/e5.moda.an.sfc.128_034_sstk.ll025sc.'
base_url3 = '.nc'

# Pandas fuction that creates an array of date time objects with an annual frequency
# during the period of interest
pr = pd.date_range(start='1979-01',end='1982-12', freq='AS')

file_list=[]
for dt in pr:
    year = dt.strftime('%Y')
    month = dt.strftime('%Y%m%d%H')
    month2 = (dt + pd.DateOffset(months=11)).strftime('%Y%m%d%H')

    # Builds complete file name and appends it to a list of files that need to
    # be downloaded
    single_file=(base_url+year+base_url2+month+'_'+month2+base_url3)
    file_list.append(single_file)


In [21]:
file_list

['https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/1979/e5.moda.an.sfc.128_034_sstk.ll025sc.1979010100_1979120100.nc',
 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/1980/e5.moda.an.sfc.128_034_sstk.ll025sc.1980010100_1980120100.nc',
 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/1981/e5.moda.an.sfc.128_034_sstk.ll025sc.1981010100_1981120100.nc',
 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/1982/e5.moda.an.sfc.128_034_sstk.ll025sc.1982010100_1982120100.nc']

In [23]:
pacific_north_lat = 65.0
pacific_south_lat = -65.0
pacific_east_lon = 120.0
pacific_west_lon = 300.0

ds_sst = xr.open_mfdataset(file_list, parallel=True, chunks={'time': '50MB'}).sel(
    latitude=slice(pacific_north_lat,pacific_south_lat,1), longitude=slice(pacific_east_lon, pacific_west_lon,1))
# v_250_hr12_ds = ds_v['V'].sel(time=datetime.time(12))

# # Extract the long term mean over the 1981-2010 period.
# ds['SSTK'].sel(latitude=slice(5,-5),longitude=slice(360-170,360-120)).mean(dim=['latitude','longitude'])
# mean_1981_2010 = ds_sub.sel(time=slice('1981-1-1','2010-12-31')).mean()

# mean_1981_2010.compute()

In [38]:
ds_sst['SSTK']

<xarray.DataArray 'SSTK' (time: 48, latitude: 521, longitude: 721)>
dask.array<getitem, shape=(48, 521, 721), dtype=float32, chunksize=(12, 521, 721), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 65.0 64.75 64.5 64.25 ... -64.5 -64.75 -65.0
  * longitude  (longitude) float64 120.0 120.2 120.5 120.8 ... 299.5 299.8 300.0
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 ... 1982-12-01
Attributes:
    long_name:                     Sea surface temperature
    short_name:                    sstk
    units:                         K
    original_format:               WMO GRIB 1 with ECMWF local table
    ecmwf_local_table:             128
    ecmwf_parameter:               34
    minimum_value:                 269.3069
    maximum_value:                 306.40942
    grid_specification:            0.25 degree x 0.25 degree from 90N to 90S ...
    rda_dataset:                   ds633.1
    rda_dataset_url:               https:/rda.ucar.edu/datasets/ds633.1/
    rda_dataset_doi:               DOI: 10.5065/P8GT-0R61
    rda_dataset_group:             ERA5 monthly mean atmospheric surface anal...
    number_of_significant_digits:  7
    _ChunkSizes:                   [  3 389 776]

In [51]:
# 
monthly_mean_sst = ds_sst["SSTK"].groupby([ds_sst.utc_date.month]).mean(dim=['latitude','longitude']).compute()

AttributeError: 'DataArray' object has no attribute 'month'

In [47]:
monthly_mean_sst

<xarray.DataArray 'SSTK' (time: 48)>
array([291.09103, 291.12936, 291.08798, 290.9861 , 290.91415, 291.11642,
       291.42584, 291.6728 , 291.59195, 291.22614, 291.03708, 291.0478 ,
       291.2155 , 291.2398 , 291.24124, 291.06476, 290.96094, 291.1612 ,
       291.38242, 291.55502, 291.45486, 291.08377, 290.85837, 290.87567,
       290.94324, 291.01068, 290.98056, 290.9853 , 291.07022, 291.1669 ,
       291.44308, 291.57846, 291.58948, 291.2815 , 290.93262, 290.86575,
       291.06097, 291.11545, 291.09686, 291.0748 , 291.01614, 291.09686,
       291.32608, 291.58868, 291.51392, 291.2697 , 291.1016 , 291.12375],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 1982-12-01